In [ ]:
pip install pandas==1.1.0

In [ ]:
import datetime
import pandas as pd
pd.set_option('display.max_columns', None, 'display.max_rows', None)
pd.options.mode.chained_assignment = None

kf_time_SV_initial = pd.read_excel("время св на точках.xlsx")
kf_time_SV_initial.columns = kf_time_SV_initial.columns.str.lower()
kf_time_SV_initial.columns = kf_time_SV_initial.columns.str.strip()

kf_time_SV_data_initial = pd.read_excel("отчет для расчета.xlsx")
kf_time_SV_data_initial.columns = kf_time_SV_data_initial.columns.str.lower()
kf_time_SV_data_initial.columns = kf_time_SV_data_initial.columns.str.strip()
kf_time_SV_data_initial['торговое предприятие'] = kf_time_SV_data_initial['торговое предприятие'].fillna(method='ffill')


cookers_initial = pd.read_excel("кол-во поваров на точке.xlsx")
cookers_initial.columns = cookers_initial.columns.str.lower()
cookers_initial.columns = cookers_initial.columns.str.strip()

df_concat_SV = pd.DataFrame()

#------------------------------
def samovivoz(x):
    index_iloc=0
    #return 0
    for i in range(kf_time_SV['время Minutes'].count()):
        if (kf_time_SV['metka'].iloc[i]==False) and \
        x['время открытия Minutes'] >= kf_time_SV['время Minutes'].iloc[i] and \
        x['время открытия Minutes'] < kf_time_SV['время Minutes'].iloc[i+1] and \
        x['время открытия'].month == kf_time_SV['дата'].iloc[i].month and \
        x['время открытия'].year == kf_time_SV['дата'].iloc[i].year and \
        x['время открытия'].day == kf_time_SV['дата'].iloc[i].day:
            return int(i)
        elif (kf_time_SV['metka'].iloc[i]==True) and \
        x['время открытия Minutes'] >= kf_time_SV['время Minutes'].iloc[i] and \
        x['время открытия'].month == kf_time_SV['дата'].iloc[i].month and \
        x['время открытия'].year == kf_time_SV['дата'].iloc[i].year and \
        x['время открытия'].day == kf_time_SV['дата'].iloc[i].day:
            return int(i)
        else:
            continue

#------------------------------                  
def max_time(x):
    time_pizza = 0
    time_goriachka = 0
    time_rolly = 0
    
    if (x['пиццы']==0 or pd.isna(x['пиццы'])):
        time_pizza = 0
    else:
        time_pizza = kf_time_SV['пиццы'].iloc[x['new']]
    
    if (x['горячка']==0 or pd.isna(x['горячка'])):
        time_goriachka = 0
        
    else:
        time_goriachka = kf_time_SV['горячка'].iloc[x['new']]
        
    if (x['роллы']==0 or pd.isna(x['роллы'])):
        time_rolly = 0
    else:
        time_rolly = kf_time_SV['роллы'].iloc[x['new']]
       
    number_list=[time_pizza, time_goriachka, time_rolly]
    #print(number_list)
    max_value = max(number_list)
    max_index = number_list.index(max_value)
    #print(max_index)
   
    if max_index==0:
        return time_pizza
    
    if max_index==1:
        return time_goriachka
    
    if max_index==2:
        return time_rolly

#------------------------------

Tpoints=[ 'Горького','Раздольная','Алроса']
for i in Tpoints:
    kf_time_SV = kf_time_SV_initial[(kf_time_SV_initial['точка'] == i)]
    kf_time_SV_data = kf_time_SV_data_initial[(kf_time_SV_data_initial['торговое предприятие'] == i)]
    cookers = cookers_initial[(cookers_initial['точка'] == i)]                       
    
    kf_time_SV_data.reset_index(drop=True)
    kf_time_SV = kf_time_SV.reset_index(drop=True)
    cookers = cookers.reset_index(drop=True)
   
    kf_time_SV['время Minutes'] = kf_time_SV.apply(lambda x: 60*x['время'].hour + x['время'].minute, axis=1)
    kf_time_SV['дата']=kf_time_SV.apply(lambda x: x['дата'].date(), axis=1)
    kf_time_SV['metka']=False
    

    #------------------------------
   
    for i in range(kf_time_SV['время Minutes'].count()-1):
        if kf_time_SV['дата'].iloc[i]<kf_time_SV['дата'].iloc[i+1]:
            kf_time_SV['metka'].iloc[i]=True

    #------------------------------   
    kf_time_SV['metka'].iloc[kf_time_SV['время Minutes'].count()-1]=True
    kf_time_SV_data['время открытия Minutes'] = kf_time_SV_data.apply(lambda x: 60*x['время открытия'].hour + x['время открытия'].minute + x['время открытия'].second/60, axis=1)
    kf_time_SV_data['new'] = kf_time_SV_data.apply(samovivoz, axis=1)
    #kf_time_SV_data.to_excel('itog1111.xlsx')
    kf_time_SV_data['время заявленное кухней'] = kf_time_SV_data.apply(max_time, axis=1)
   
    
    cookers['Hour'] = cookers.apply(lambda x: x['час'].hour, axis=1)
    cookers['Date'] = cookers['дата'].dt.day
    cookers['Month'] = cookers['дата'].dt.month
    kf_time_SV_data['Date']=kf_time_SV_data['время открытия'].dt.day
    kf_time_SV_data['Month']=kf_time_SV_data['время открытия'].dt.month
    kf_time_SV_data['Hour']=kf_time_SV_data['время открытия'].dt.hour
    kf_time_SV_data.head()
    kf_time_SV_data['Количество поваров'] = 0
   
    
    for i in range(cookers.shape[0]):
        kf_time_SV_data.loc[(kf_time_SV_data['Hour'] == cookers['Hour'][i]) & (kf_time_SV_data['Month'] == cookers['Month'][i]) & (kf_time_SV_data['Date'] == cookers['Date'][i]), 'Количество поваров'] = cookers['кол-во поваров'][i]

    df_concat_SV = pd.concat([df_concat_SV, kf_time_SV_data], axis=0)


df_concat_SV.to_excel('itog.xlsx')

In [ ]:
import datetime 
from datetime import date, datetime, time, timedelta
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)

kf_logostic = pd.read_excel("itog.xlsx")
kf_logostic.columns = kf_logostic.columns.str.lower()
kf_logostic.columns = kf_logostic.columns.str.strip()

def f(x):
    if (x['Опоздание доставка общ'] >= 0):
        return 0
    else:
        return 1
    
def f1(x):
    if (x['Опоздание кухня'] >= 0):
        return 0
    else:
        return 1
    
def f2(x):
    if (x['Опоздание кухня'] < 0):
        return x['Опоздание доставка общ'] - x['Опоздание кухня']
    else:
        return x['Опоздание доставка общ']
    
    
def f3(x):
    if (x['Опоздание только курьеры'] >= 0):
        return 0
    else:
        return 1 

def f4(x):
    if pd.isna(x['источник заказа']):
        return 'kvikfud.ru'
    else:
        return x['источник заказа'] 

kf_logostic['Время доставки'] = kf_logostic['фактическое время доставки'] - kf_logostic['время открытия']
kf_logostic['Время доставки'] = kf_logostic['Время доставки'].dt.total_seconds()/60

kf_logostic['Опоздание доставка общ'] = kf_logostic['планируемое время доставки'] - kf_logostic['фактическое время доставки']
kf_logostic['Опоздание доставка общ'] = kf_logostic['Опоздание доставка общ'].dt.total_seconds()/60

kf_logostic['озвучено время курьерами'] = kf_logostic['планируемое время доставки'] - kf_logostic['время открытия']
kf_logostic['озвучено время курьерами'] = kf_logostic['озвучено время курьерами'].dt.total_seconds()/60 - kf_logostic['время заявленное кухней']

kf_logostic['озвучено время курьерами'] = kf_logostic['озвучено время курьерами'].round(decimals=-1)

kf_logostic['Флаг опоздания общ'] = kf_logostic.apply(f, axis=1)
kf_logostic['Флаг опоздания общ'].value_counts()

kf_logostic['Время приготовления'] = kf_logostic['время печати накладной'] - kf_logostic['время открытия']
kf_logostic['Время приготовления'] = kf_logostic['Время приготовления'].dt.total_seconds()/60

kf_logostic['Опоздание кухня'] = kf_logostic['время заявленное кухней']- kf_logostic['Время приготовления']

kf_logostic['Флаг опоздания кухня'] = kf_logostic.apply(f1, axis=1)
kf_logostic['Флаг опоздания кухня'].value_counts()

kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['заказы не для учета'] == 1].index)

kf_deep= kf_logostic.copy()
kf_deep['Опоздание только курьеры'] = kf_deep.apply(f2, axis=1)
kf_deep['Флаг опоздание только курьеры'] = kf_deep.apply(f3, axis=1)

kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['тип заказа'] == 'На время самовывоз'].index)
kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['тип заказа'] == 'На время курьером'].index)

print('Доставлено заказов вовремя: ', kf_logostic['Флаг опоздания общ'].value_counts()[0])
print('Доставлено заказов с опозданием: ', kf_logostic['Флаг опоздания общ'].value_counts()[1])
print('Процент опозданий заказов: ', round(100*kf_logostic['Флаг опоздания общ'].value_counts()[1]/(kf_logostic['Флаг опоздания общ'].value_counts()[1]+kf_logostic['Флаг опоздания общ'].value_counts()[0]),2),'%')
print('\n')

print('Приготовлено вовремя заказов: ', kf_logostic['Флаг опоздания кухня'].value_counts()[0])
print('Опозданий в приготовлении: ', kf_logostic['Флаг опоздания кухня'].value_counts()[1])
print('Процент опозданий в приготовлении: ', round(100*kf_logostic['Флаг опоздания кухня'].value_counts()[1]/(kf_logostic['Флаг опоздания кухня'].value_counts()[1]+kf_logostic['Флаг опоздания кухня'].value_counts()[0]),2),'%')
print('\n')

kf_logostic_without_bad_closed = kf_logostic.drop(kf_logostic[kf_logostic['заказы не для учета'] == 1].index)
kf_logostic_without_bad_closed = kf_logostic_without_bad_closed.drop(kf_logostic_without_bad_closed[kf_logostic_without_bad_closed['тип заказа'] == 'Доставка самовывоз'].index)
kf_logostic_without_bad_closed['Опоздание только курьеры'] = kf_logostic_without_bad_closed.apply(f2, axis=1)
kf_logostic_without_bad_closed['Флаг опоздание только курьеры'] = kf_logostic_without_bad_closed.apply(f3, axis=1)

print('Доставлено курьерами вовремя: ', kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[0])
print('Опозданий по вине курьеров: ', kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1])
print('Процент опозданий по вине курьеров: ', round(100*kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1]/(kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1]+kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[0]),2),'%')
print('\n')

#kf_logostic_without_bad_closed['время заявленное кухней'].value_counts()
kf_logostic_without_bad_closed['источник заказа'] = kf_logostic_without_bad_closed.apply(f4, axis=1)
kf_logostic_without_delivery = kf_logostic_without_bad_closed.drop(kf_logostic_without_bad_closed[kf_logostic_without_bad_closed['источник заказа'] == 'delivery-club.ru'].index)
print(kf_logostic_without_delivery.groupby('озвучено время курьерами')['источник заказа'].value_counts())

kf_logostic_without_delivery.loc[(kf_logostic_without_delivery['озвучено время курьерами']>70) | (kf_logostic_without_delivery['озвучено время курьерами']<20)].head(10)

otchet=kf_logostic_without_delivery.loc[(kf_logostic_without_delivery['озвучено время курьерами']>70) | (kf_logostic_without_delivery['озвучено время курьерами']<20)]
otchet.to_excel('проблемные заказы.xlsx')

In [ ]:
import math
from openpyxl import load_workbook

colomns_for_additional=[
'торговое предприятие',
'время заявленное кухней',
'Время приготовления',
'номер чека',
'время открытия',
'время печати накладной',
'планируемое время доставки',
'время отправки доставки',
'фактическое время доставки',
'город',
'адрес',
'оператор доставки',
'тип заказа',
'источник заказа',
'курьер']

writer = pd.ExcelWriter('additional.xlsx', engine='openpyxl')
writer.book = load_workbook('additional.xlsx')   
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'additional.xlsx', sheet_name='Долгое приготовление')
kf_deep[kf_deep['Время приготовления']!=kf_deep['Время приготовления']][colomns_for_additional].to_excel(writer, sheet_name='Долгое приготовление', index=False, header=True, startrow=len(reader)+1)
writer.close() 


kf_deep['Day of Week'] = kf_deep['время открытия'].dt.dayofweek
kf_deep['Day of Week'].replace({0: "ПН", 1: "ВТ", 2: "СР", 3: "ЧТ", 4: "ПТ", 5: "СБ", 6: "ВС"}, inplace=True)

def conditional_time(x):
    if (x['тип заказа'] == 'На время курьером') or (x['тип заказа'] == 'На время самовывоз'):
        return x['время печати накладной'] - timedelta(minutes=25)
    else:
        return x['время открытия']

kf_deep['время открытия'] = kf_deep.apply(conditional_time, axis=1)
kf_deep['Время приготовления'] = kf_deep['время печати накладной'] - kf_deep['время открытия']
kf_deep['Время приготовления'] = kf_deep['Время приготовления'].dt.total_seconds()/60

kf_deep['Hour'] = kf_deep['время открытия'].dt.hour

def cooking_time(kf_deep_x, point, day, start, end):
    kf_deep = kf_deep_x.loc[(kf_deep_x['торговое предприятие'] == point) & (kf_deep_x['Day of Week']==day) & (kf_deep_x['Hour'] >=start) & (kf_deep_x['Hour'] <end)] 
    
    groups = kf_deep.groupby('Hour')['Hour'].value_counts()
    #print(groups)
    str_hour = ''
    for i in groups:
        str_hour = str_hour + str(i) + '/ '
      
    groups = kf_deep.groupby('Hour')['общая сумма чека'].sum()
    str_sum_hour = ''
    str_sum_hour_per_cooker = ''
    for i in groups:
        str_sum_hour = str_sum_hour + str(int(i/1000)) + '/ '
        str_sum_hour_per_cooker = str_sum_hour_per_cooker + str(int((i)/math.ceil(kf_deep['количество поваров'].mean()))) + '/ '
    
    #запись в файл отчета заказов, которые готовились дольше всего
    writer = pd.ExcelWriter('additional.xlsx', engine='openpyxl')
    writer.book = load_workbook('additional.xlsx')   
    writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
    reader = pd.read_excel(r'additional.xlsx', sheet_name='Долгое приготовление')
    kf_deep[(kf_deep['Время приготовления'] <= kf_deep['Время приготовления'].max()) & (kf_deep['Время приготовления'] >= kf_deep['Время приготовления'].max()*0.8) & (kf_deep['Время приготовления'] >= 30)][colomns_for_additional].to_excel(writer, sheet_name='Долгое приготовление', index=False, header=False, startrow=len(reader)+2)
    writer.close() 
       
    
    return kf_deep['Время приготовления'].median(), kf_deep['Время приготовления'].max(), kf_deep['Флаг опоздания кухня'].sum(), kf_deep['Флаг опоздания кухня'].count(), \
           kf_deep['общая сумма чека'].sum(), kf_deep['количество поваров'].mean(), str_hour, str_sum_hour, str_sum_hour_per_cooker

cooking_array = np.array([['Алроса', 'ПН', 10, 16],
                 ['Алроса', 'ВТ', 10, 16],
                 ['Алроса', 'СР', 10, 16],
                 ['Алроса', 'ЧТ', 10, 16],
                 ['Алроса', 'ПТ', 10, 16],
                 ['Алроса', 'СБ', 10, 16],
                 ['Алроса', 'ВС', 10, 16],
                 ['Алроса', 'ПН', 16, 22],
                 ['Алроса', 'ВТ', 16, 22],
                 ['Алроса', 'СР', 16, 22],
                 ['Алроса', 'ЧТ', 16, 22],
                 ['Алроса', 'ПТ', 16, 22],
                 ['Алроса', 'СБ', 16, 22],
                 ['Алроса', 'ВС', 16, 22],
                 ['Горького', 'ПН', 10, 16],
                 ['Горького', 'ВТ', 10, 16],
                 ['Горького', 'СР', 10, 16],
                 ['Горького', 'ЧТ', 10, 16],
                 ['Горького', 'ПТ', 10, 16],
                 ['Горького', 'СБ', 10, 16],
                 ['Горького', 'ВС', 10, 16],
                 ['Горького', 'ПН', 16, 22],
                 ['Горького', 'ВТ', 16, 22],
                 ['Горького', 'СР', 16, 22],
                 ['Горького', 'ЧТ', 16, 22],
                 ['Горького', 'ПТ', 16, 22],
                 ['Горького', 'СБ', 16, 22],
                 ['Горького', 'ВС', 16, 22],
                 ['Раздольная', 'ПН', 10, 16],
                 ['Раздольная', 'ВТ', 10, 16],
                 ['Раздольная', 'СР', 10, 16],
                 ['Раздольная', 'ЧТ', 10, 16],
                 ['Раздольная', 'ПТ', 10, 16],
                 ['Раздольная', 'СБ', 10, 16],
                 ['Раздольная', 'ВС', 10, 16],
                 ['Раздольная', 'ПН', 16, 22],
                 ['Раздольная', 'ВТ', 16, 22],
                 ['Раздольная', 'СР', 16, 22],
                 ['Раздольная', 'ЧТ', 16, 22],
                 ['Раздольная', 'ПТ', 16, 22],
                 ['Раздольная', 'СБ', 16, 22],
                 ['Раздольная', 'ВС', 16, 22]])

label = []
max_array = []
min_array = []
mean_array = []
lateness_array = []
receipts_array = []
ours_array = []
sum_array = []
cookers_array = []
orders_per_hour_array = []
str_sum_hour_array = []
str_sum_hour_per_cooker_array = []

for i in cooking_array:
    if len(kf_deep.loc[(kf_deep['торговое предприятие'] == str(i[0])) & (kf_deep['Day of Week']==str(i[1])) & (kf_deep['Hour'] >=int(i[2])) & (kf_deep['Hour'] <int(i[3]))]) == 0:
        label.append(str(i[0][:3]) + ' ' + str(i[2]) + '-' + str(i[3]) + ' ' + str(i[1]) + ' ПУСТО')
        ours_array.append(1)
        mean_array.append(1)
        max_array.append(1)
        lateness_array.append(1)
        receipts_array.append(1)
        sum_array.append(1)
        cookers_array.append(1)
        orders_per_hour_array.append(1)
        str_sum_hour_array.append(1)
        str_sum_hour_per_cooker_array.append(1)
        continue
        
    mean, max, lateness, receipts, sum, cookers, orders_per_hour, str_sum_hour, str_sum_hour_per_cooker = cooking_time(kf_deep, str(i[0]), str(i[1]), int(i[2]), int(i[3]))
    label.append(str(i[0][:3]) + ' ' + str(i[2]) + '-' + str(i[3]) + ' ' + str(i[1]) + ' ' + str(kf_deep.loc[kf_deep['Day of Week']==i[1]].iloc[0]['время открытия'].date().day) + '.' + str(kf_deep.loc[kf_deep['Day of Week']==i[1]].iloc[0]['время открытия'].date().month))
    #label.append(str(i[0]) + ' ' + str(i[2]) + '-' + str(i[3]) + ' ' + str(i[1]))
    ours_array.append(int(i[3])-int(i[2]))
    mean_array.append(int(mean))
    max_array.append(int(max))
    lateness_array.append(lateness)
    receipts_array.append(receipts)
    sum_array.append(sum)
    cookers_array.append(cookers)
    orders_per_hour_array.append(orders_per_hour)
    str_sum_hour_array.append(str_sum_hour)
    str_sum_hour_per_cooker_array.append(str_sum_hour_per_cooker)
    
df = pd.DataFrame(data=mean_array, index=label) 
df.columns =['Среднее время']
df['Максимальное'] = max_array
df['Опоздания кухни'] = lateness_array
df['Чеков всего'] = receipts_array
df['Сумма чеков'] = sum_array
df['Чеков в час'] = np.array(receipts_array)/np.array(ours_array)
df['Чеков в час'] = df['Чеков в час'].round(decimals = 0).astype(int)
df['Сумма в час'] = np.array(sum_array)/np.array(ours_array)
df['Сумма в час'] = df['Сумма в час'].round(decimals = 0).astype(int)
df['Поваров'] = cookers_array
df['Сумма на повара в час'] = df['Сумма в час']/cookers_array
df['Сумма на повара в час']=df['Сумма на повара в час'].round(decimals = 0).astype(int)
df['Поваров'] = df['Поваров'].round(decimals = 1).astype(int)
df['Заказов по часам'] = orders_per_hour_array
df['Тысячи по часам'] =  str_sum_hour_array
df['Сумма на повара по часам'] =  str_sum_hour_per_cooker_array


display(df.sort_values(by=['Среднее время'], ascending=False).style.background_gradient(cmap='Blues'))
display(df.style.background_gradient(cmap='Blues'))

df.sort_values(by=['Среднее время'], ascending=False).style.background_gradient(cmap='Blues').to_excel('cool.xlsx')

kf_deep = kf_deep.drop(kf_deep[kf_deep['тип заказа'] == 'На время самовывоз'].index)

kf_deep['Стоял на точке'] = kf_deep['время отправки доставки'] - kf_deep['время печати накладной'] 
kf_deep['Стоял на точке'] = kf_deep['Стоял на точке'].dt.total_seconds()/60
bins = [-1000, 0, 10, 20, 30, 40, 50, 60, 1000]
labels = ['<0!','<10', '10-20', '20-30', '30-40', '40-50', '50-60', '>60']
cats = pd.cut(kf_deep['Стоял на точке'], bins, labels=labels)
pd.value_counts(cats)
kf_deep['Bins']=cats

def bins(kf_deep_x, point):
    kf_deep = kf_deep_x.loc[kf_deep_x['торговое предприятие'] == point] 
    return kf_deep['Bins'].value_counts()  

def bins_day(kf_deep_x, point, day, start, end):
    kf_deep = kf_deep_x.loc[(kf_deep_x['торговое предприятие'] == point) & (kf_deep_x['Day of Week']==day) & (kf_deep_x['Hour'] >=start) & (kf_deep_x['Hour'] <end)] 
    return kf_deep['Bins'].value_counts()

def table_day(kf_deep_x, point="всем точкам"):
    if point=="всем точкам":
        kf_deep = kf_deep_x
    else:
        kf_deep = kf_deep_x.loc[(kf_deep_x['торговое предприятие'] == point)]
    print('Данные по', point, 'Сколько заказов ожидали отправки')
    grouped = kf_deep.groupby('Day of Week').Bins.value_counts().unstack(0)[["ПН", "ВТ", "СР", "ЧТ", "ПТ", "СБ", "ВС"]].fillna(0.0).astype(int)
    grouped['Итого'] = grouped.sum(axis=1) 
    print(grouped, '\n')
#    print(kf_deep.head(), '\n')
#    grouped[3:].plot(kind='bar', title = "Данные по " + point, xlabel='Время ожидания на точке', ylabel='Количество заказов').get_figure().savefig(str(point)+'.png')
    return kf_deep['Bins'].value_counts()

final_table_amount_orders = pd.DataFrame(index=['<10', '10-20', '20-30', '30-40', '40-50', '50-60', '>60'])
final_table_amount_orders=final_table_amount_orders.join(bins(kf_deep, 'Алроса'))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Алр всего'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Алроса', "ПТ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Алр Fri 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Алроса', "СБ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Алр Sat 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins(kf_deep, 'Горького'))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Гор всего'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Горького', "ПТ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Гор Fri 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Горького', "СБ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Гор Sat 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins(kf_deep, 'Раздольная'))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Разд всего'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Раздольная', "ПТ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Разд Fri 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Раздольная', "СБ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Разд Sat 17-20'})

table_day(kf_deep)
table_day(kf_deep, 'Алроса')
table_day(kf_deep, 'Горького')
table_day(kf_deep, 'Раздольная')

In [ ]:
import datetime 
from datetime import date, datetime, time, timedelta
import pandas as pd
import numpy as np 
first = 0

colomns_for_additional=[
'торговое предприятие',
'Стоял на точке',
'номер чека',
'время открытия',
'время печати накладной',
'время отправки доставки',
'планируемое время доставки',
'фактическое время доставки',
'курьер',
'город',
'адрес',
'оператор доставки',
'тип заказа',
'источник заказа',
'общая сумма чека']

pd.set_option('display.float_format','{:.2f}'.format)
pd.set_option("display.max_rows", None,'display.max_columns', None)
pd.set_option('display.max_rows', None)

deliveryman = kf_deep
#adress_control = kf_deep.copy()
deliveryman['курьер'].unique()
deliveryman['курьер'].value_counts()
deliveryman['День'] = deliveryman['время открытия'].dt.dayofweek 
deliveryman['День'].replace({0: "ПН", 1: "ВТ", 2: "СР", 3: "ЧТ", 4: "ПТ", 5: "СБ", 6: "ВС"}, inplace=True)
deliveryman['Время'] = deliveryman['время отправки доставки'].dt.strftime('%H:%M')

def order_amount(x):
    
    a = deliveryman.loc[(deliveryman['курьер'] == x['курьер']) & (deliveryman['время отправки доставки'] > (x['время отправки доставки']) - timedelta(minutes=5)) \
                        & (deliveryman['время отправки доставки'] < (x['время отправки доставки']) + timedelta(minutes=5)) \
                        & (deliveryman['День']==x['День']) & (deliveryman['торговое предприятие']==x['торговое предприятие'])].shape[0]
    return a

df_concat = pd.DataFrame(columns=['время отправки доставки','курьер', 'Заказов в рейс'])

for dm in deliveryman['курьер'].unique():
    deliveryman['Заказов в рейс'] = deliveryman.loc[(deliveryman['курьер'] == dm)].apply(order_amount, axis=1)
    df_concat = pd.concat([df_concat, deliveryman.loc[(deliveryman['курьер'] == dm)]], axis=0)

for_print = df_concat[df_concat['Заказов в рейс'] > 3].sort_values(by=['Заказов в рейс'], ascending=False).sort_values(by=['время отправки доставки'], ascending=False)[['курьер', 'Заказов в рейс','Время','торговое предприятие','День']]

#print(for_print[for_print['торговое предприятие']=='Алроса'].sort_values(by=['Время'], ascending=False).reset_index(drop=True), '\n')
#print(for_print[for_print['торговое предприятие']=='Горького'].sort_values(by=['Время'], ascending=False).reset_index(drop=True), '\n')
#print(for_print[for_print['торговое предприятие']=='Раздольная'].sort_values(by=['Время'], ascending=False).reset_index(drop=True), '\n')

df_concat[df_concat['Заказов в рейс'] > 3].sort_values(by=['Заказов в рейс'], ascending=False).sort_values(by=['время отправки доставки'], ascending=False).to_excel('рейсы.xlsx')

pivot_table_DM = pd.pivot_table(df_concat[['date', 'День', 'торговое предприятие','курьер']], index = ['курьер','День', 'торговое предприятие'], aggfunc='count')

orders = df_concat.groupby('курьер')['Заказов в рейс'].value_counts()
orders= orders.to_frame()
orders=orders.rename(columns={"Заказов в рейс": "Количество рейсов"})
orders=orders.reset_index()
orders=orders.rename(columns={"Заказов в рейс": "Заказов в рейсе"})
orders['Количество рейсов'] = orders['Количество рейсов']/orders['Заказов в рейсе']
orders = orders.astype({'Количество рейсов': 'int32','Заказов в рейсе': 'int32'})


full_table = orders.groupby('Заказов в рейсе')['Количество рейсов'].sum().sort_index(ascending=False)
full_table=full_table.to_frame()
full_table=full_table.reset_index()
#full_table

#таблица по курьерам
df_concat['bins_weight'] = df_concat['Bins'] 
df_concat['bins_weight'].replace({'<0!': 0,'<10': 0, '10-20': 0, '20-30': 0, '30-40' : 1, '40-50': 2, '50-60': 3, '>60': 4}, inplace=True)

DM_array = np.array([['Алроса', 'ПН', 10, 16],
                 ['Алроса', 'ВТ', 10, 16],
                 ['Алроса', 'СР', 10, 16],
                 ['Алроса', 'ЧТ', 10, 16],
                 ['Алроса', 'ПТ', 10, 16],
                 ['Алроса', 'СБ', 10, 16],
                 ['Алроса', 'ВС', 10, 16],
                 ['Алроса', 'ПН', 16, 22],
                 ['Алроса', 'ВТ', 16, 22],
                 ['Алроса', 'СР', 16, 22],
                 ['Алроса', 'ЧТ', 16, 22],
                 ['Алроса', 'ПТ', 16, 22],
                 ['Алроса', 'СБ', 16, 22],
                 ['Алроса', 'ВС', 16, 22],
                 ['Горького', 'ПН', 10, 16],
                 ['Горького', 'ВТ', 10, 16],
                 ['Горького', 'СР', 10, 16],
                 ['Горького', 'ЧТ', 10, 16],
                 ['Горького', 'ПТ', 10, 16],
                 ['Горького', 'СБ', 10, 16],
                 ['Горького', 'ВС', 10, 16],
                 ['Горького', 'ПН', 16, 22],
                 ['Горького', 'ВТ', 16, 22],
                 ['Горького', 'СР', 16, 22],
                 ['Горького', 'ЧТ', 16, 22],
                 ['Горького', 'ПТ', 16, 22],
                 ['Горького', 'СБ', 16, 22],
                 ['Горького', 'ВС', 16, 22],
                 ['Раздольная', 'ПН', 10, 16],
                 ['Раздольная', 'ВТ', 10, 16],
                 ['Раздольная', 'СР', 10, 16],
                 ['Раздольная', 'ЧТ', 10, 16],
                 ['Раздольная', 'ПТ', 10, 16],
                 ['Раздольная', 'СБ', 10, 16],
                 ['Раздольная', 'ВС', 10, 16],
                 ['Раздольная', 'ПН', 16, 22],
                 ['Раздольная', 'ВТ', 16, 22],
                 ['Раздольная', 'СР', 16, 22],
                 ['Раздольная', 'ЧТ', 16, 22],
                 ['Раздольная', 'ПТ', 16, 22],
                 ['Раздольная', 'СБ', 16, 22],
                 ['Раздольная', 'ВС', 16, 22]])

def deliverymen_KPI(kf_deep_x, point, day, start, end):
    global first
    kf_deep = kf_deep_x.loc[(kf_deep_x['торговое предприятие'] == point) & (kf_deep_x['Day of Week']==day) & (kf_deep_x['Hour'] >=start) & (kf_deep_x['Hour'] <end) & (kf_deep_x['курьер']!='Самовывоз')]
    
    if kf_deep.size > 0:
        groups = kf_deep.groupby('Hour')['Hour'].value_counts()
    else:
        groups = []
    str_hour = ''
    str_DM = ''
    for i in range(len(kf_deep['курьер'].unique())):
        #print(kf_deep['Курьер'].unique()[i].split()[0])
        str_DM = str_DM + str(kf_deep.loc[kf_deep['курьер']==kf_deep['курьер'].unique()[i]].shape[0]) + ' ' + str(kf_deep['курьер'].unique()[i].split()[0]) + ' ' + \
        str(kf_deep.loc[kf_deep['курьер']==kf_deep['курьер'].unique()[i]]['время отправки доставки'].sort_values(ascending=True)[:1].values[0].astype('datetime64[m]'))[-5:] + '-' +\
        str(kf_deep.loc[kf_deep['курьер']==kf_deep['курьер'].unique()[i]]['время отправки доставки'].sort_values(ascending=True)[-1:].values[0].astype('datetime64[m]'))[-5:] + ' | '
    for i in groups:
        str_hour = str_hour + str(i) + '/'
        
    temp = (kf_deep.groupby(['Заказов в рейс']).size()/list(kf_deep.groupby(['Заказов в рейс']).size().index)).apply(np.ceil)
    
    reses=''
    for i in range(temp.shape[0]):
        if ((temp.index[i] >= 1) & (temp.index[i] <= 3)):
            reses= reses + str(int(temp.index[i])) + ': ' + str(int(temp.iloc[i])) + '/'
            
    reses4=''
    for i in range(temp.shape[0]):
        if temp.index[i] >= 4:
            reses4= reses4 + str(int(temp.index[i])) + ': ' + str(int(temp.iloc[i])) + '/'
    
    if first==0:
        header_true = True
        first=1
    else:
        header_true = False
    
    dfd = kf_deep['Bins'].value_counts()
    bins_ = '| '.join(str(dfd[(dfd>0) & (dfd.index!='<10')&(dfd.index!='10-20')&(dfd.index!='20-30')].sort_index()).split('\n')[:-1]).replace('    ', ' ')
    
    
    writer = pd.ExcelWriter('additional.xlsx', engine='openpyxl')
    writer.book = load_workbook('additional.xlsx')   
    writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
    reader = pd.read_excel(r'additional.xlsx', sheet_name='Простой заказов на точке')
    kf_deep[kf_deep['bins_weight']>0][colomns_for_additional].to_excel(writer, sheet_name='Простой заказов на точке', index=False, header=header_true, startrow=len(reader)+2)
    writer.close() 
    
    return kf_deep['bins_weight'].sum(),  bins_, kf_deep['bins_weight'].count(), len(kf_deep['курьер'].unique()), str_hour, str_DM, kf_deep['Флаг опоздание только курьеры'].sum(), kf_deep['Флаг опоздания общ'].sum(), reses, reses4,  kf_deep['время в пути(мин)'].mean()


delivery_counts_array = []
bins_array = []
counts_order_array = []
counts_of_deliverymen = []
orders_per_hour_array = []
label = []
deliverymenFIO_array = []
delay_by_DM_array = [] 
delay_comman_array = [] 
reses_DM_array = [] 
reses4_DM_array = [] 
time_road_array = [] 
day_array = []
hour_array = []
point_array = []
   

for i in DM_array:
    delivery_counts,  bins_, counts_order, deliverymen_counts, orders_per_hour, deliverymenFIO, delay_by_DM, delay_comman, reses_DM, reses4_DM, time_road = deliverymen_KPI(df_concat, str(i[0]), str(i[1]), int(i[2]), int(i[3]))
    label.append(str(i[0][:3]) + ' ' + str(i[2]) + '-' + str(i[3]) + ' ' + str(i[1]) + ' ' + str(df_concat.loc[df_concat['Day of Week']==i[1]].iloc[0]['время открытия'].date().day) + '.' + str(df_concat.loc[df_concat['Day of Week']==i[1]].iloc[0]['время открытия'].date().month))
    bins_array.append(bins_)
    orders_per_hour_array.append(orders_per_hour)
    counts_order_array.append(counts_order)
    delivery_counts_array.append(delivery_counts)
    counts_of_deliverymen.append(deliverymen_counts)
    deliverymenFIO_array.append(deliverymenFIO)
    delay_by_DM_array.append(int(delay_by_DM))
    delay_comman_array.append(int(delay_comman))
    reses_DM_array.append(reses_DM)
    reses4_DM_array.append(reses4_DM)
    time_road_array.append(time_road)
    day_array.append(str(i[1]))
    hour_array.append(int(i[2]))
    point_array.append(str(i[0]))
    
df_delivery = pd.DataFrame(data=delivery_counts_array, index=label)
df_delivery.columns = ['Простой заказов']
df_delivery['Простой'] = bins_array
df_delivery['Ср. время в пути'] = np.nan_to_num(time_road_array)
df_delivery['Ср. время в пути'] = df_delivery['Ср. время в пути'].round(decimals = 0).astype(int)
df_delivery['Заказов'] = counts_order_array 
df_delivery['Опоздния курьеры'] = delay_by_DM_array
df_delivery['Опоздния вообще'] = delay_comman_array
df_delivery['Курьеров'] = counts_of_deliverymen 
df_delivery['заказов на курьера'] = df_delivery['Заказов']/df_delivery['Курьеров']
df_delivery['заказов на курьера']= df_delivery['заказов на курьера']
df_delivery['Планирование'] = (df_delivery['Курьеров'] / df_delivery['заказов на курьера'])
df_delivery['Заказов <4'] = reses_DM_array
df_delivery['Заказов >4'] = reses4_DM_array
df_delivery['Заказов по часам'] = orders_per_hour_array
df_delivery['Курьеры'] =  deliverymenFIO_array


display(df_delivery.sort_values(by=['Простой заказов'], ascending=False).style.format(formatter={'заказов на курьера': "{:.4f}", 'Планирование': "{:.4f}"}).background_gradient(cmap='Blues'))
display(df_delivery.style.background_gradient(cmap='Blues'))
df_delivery.sort_values(by=['Простой заказов'], ascending=False).style.format(formatter={'заказов на курьера': "{:.4f}", 'Планирование': "{:.4f}"}).background_gradient(cmap='Blues').to_excel('cool2.xlsx')
